# Simple LangChain RAG.  

Following this tutorial
[Q&A with RAG](https://python.langchain.com/docs/use_cases/question_answering/)

## Improt Environement Variables from File
Need to have a .env file with OPENAI_API_KEY and a LANGSMITH_API_KEY

In [ ]:
from environs import Env
import os
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

env = Env()
env.read_env("/Users/geoffreysmalling/development/langchain/.env")

## connect to OpenAI

In [ ]:
llm = ChatOpenAI(api_key=env.str("OPENAI_API_KEY"), model="gpt-3.5-turbo-0125")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = env.str("LANGSMITH_API_KEY")



## Test llm connection and get answer

In [ ]:
llm.invoke("how can langsmith help with testing?")

# load, chuck, and index contents of the source
use the WebBaseLoader

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
print("type of docs object: " + str(type(docs)))
print("number of docs: " + str(len(docs)))


## split the docs into chunks


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print("type of splits object: " + str(type(splits)))
print("number of splits: " + str(len(splits)))
print("metadata example: " + str(splits[10].metadata))

In [ ]:
## store in a vector store using openAI Embeddings model

In [ ]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
print("type of vectorstore object: " + str(type(vectorstore)))

In [ ]:
## create a retriever and pull a prompt from the langsmith hub

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")
print(len(retrieved_docs))
print(retrieved_docs[0].page_content)
print(retrieved_docs[0].metadata['source'])


prompt = hub.pull("rlm/rag-prompt")

In [ ]:
### create a method to break each chunk of docs into a new paragraph for the prompt context

In [ ]:
def format_docs(docs):
    context =  "\n\n".join(doc.page_content + " source: " + doc.metadata['source'] + " "   for doc in docs)
    return context
    

## Build the LangChain
- User the retriever to get context docs and then join them with format docs
- Get the question to the llm via RunnablePassthrough
- pass context and question to the prompt
- pass prompt to llm
- parse results to a string, vs a message

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    #| StrOutputParser()
)

In [ ]:
response = rag_chain.invoke("What is Task Decomposition?")

In [ ]:
print(response)

# adding chat history


In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)




In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

In [ ]:
for document in ai_msg_2["context"]:
    print(document)
    print()